# Métrica 2_1_03 — Pareja y/o dependientes con empleo sin cuentas o inversiones

In [3]:
import traceback
from pymongo import MongoClient, UpdateOne
from pymongo.errors import ConnectionFailure, OperationFailure, BulkWriteError
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

METRIC_ID = "2_1_03_PAREJA_DEPENDIENTES_EMPLEO_SIN_INVERSIONES"

# --- FUNCIONES AUXILIARES ---

def tiene_pareja_con_empleo(doc):
    try:
        pareja = (
            doc.get("declaracion", {})
               .get("situacionPatrimonial", {})
               .get("datosPareja", {})
        )
        actividad = pareja.get("actividadLaboral", "")
        if isinstance(actividad, str) and actividad.strip() != "":
            return True
        return False
    except Exception:
        return False

def tiene_dependientes_con_empleo(doc):
    try:
        dependientes = (
            doc.get("declaracion", {})
               .get("situacionPatrimonial", {})
               .get("datosDependientesEconomicos", {})
               .get("dependienteEconomico", [])
        )
        if not isinstance(dependientes, list):
            return False
        for d in dependientes:
            if d.get("actividadLaboral", ""):
                return True
        return False
    except Exception:
        return False

def tiene_inversiones(doc):
    try:
        inversiones = (
            doc.get("declaracion", {})
               .get("situacionPatrimonial", {})
               .get("inversiones", {})
               .get("inversion", [])
        )
        return isinstance(inversiones, list) and len(inversiones) > 0
    except Exception:
        return False


# --- EVALUACIÓN PRINCIPAL ---

def evaluar_metrica(doc):
    pareja_empleo = tiene_pareja_con_empleo(doc)
    dep_empleo = tiene_dependientes_con_empleo(doc)
    inv = tiene_inversiones(doc)

    # Caso 1: No hay pareja o dependientes con empleo → no aplica
    if not pareja_empleo and not dep_empleo:
        return "N/A"

    # Caso 2: Hay empleo y hay inversiones → cumple
    if (pareja_empleo or dep_empleo) and inv:
        return "CUMPLE"

    # Caso 3: Hay empleo pero no hay inversiones → no cumple
    if (pareja_empleo or dep_empleo) and not inv:
        return "NO_CUMPLE"

    # Caso 4: Faltan datos en campos obligatorios → sin dato
    return "SIN_DATO"


# --- PROCESAMIENTO POR LOTES ---

def procesar_metrica_2_1_03():
    client = None
    operaciones = []
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0, "N/A": 0}
    total = 0

    try:
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]
        source = db[SOURCE_COLLECTION_NAME]
        target = db[METRICS_COLLECTION_NAME]

        print(f"Procesando métrica {METRIC_ID}...")

        cursor = source.find({}, {
            "_id": 1,
            "declaracion.situacionPatrimonial.datosPareja.actividadLaboral": 1,
            "declaracion.situacionPatrimonial.datosDependientesEconomicos.dependienteEconomico.actividadLaboral": 1,
            "declaracion.situacionPatrimonial.inversiones.inversion": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1
            resultado = evaluar_metrica(doc)
            resultados[resultado] += 1

            operaciones.append(UpdateOne(
                {"_id": doc["_id"]},
                {"$set": {METRIC_ID: resultado}},
                upsert=True
            ))

            if len(operaciones) == 1000:
                target.bulk_write(operaciones)
                operaciones.clear()
                print(f"  > Procesados {total} documentos...")

        if operaciones:
            target.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print(f"Documentos procesados: {total}")
        for k, v in resultados.items():
            print(f"  > {k}: {v}")

    except Exception as e:
        print(f"Error general: {e}")
        traceback.print_exc()
    finally:
        if client:
            client.close()
            print("Conexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_2_1_03()


Procesando métrica 2_1_03_PAREJA_DEPENDIENTES_EMPLEO_SIN_INVERSIONES...
  > Procesados 1000 documentos...
  > Procesados 2000 documentos...
  > Procesados 3000 documentos...
  > Procesados 4000 documentos...
  > Procesados 5000 documentos...
  > Procesados 6000 documentos...
  > Procesados 7000 documentos...
  > Procesados 8000 documentos...
  > Procesados 9000 documentos...
  > Procesados 10000 documentos...
  > Procesados 11000 documentos...
  > Procesados 12000 documentos...
  > Procesados 13000 documentos...
  > Procesados 14000 documentos...
  > Procesados 15000 documentos...
  > Procesados 16000 documentos...
  > Procesados 17000 documentos...
  > Procesados 18000 documentos...
  > Procesados 19000 documentos...
  > Procesados 20000 documentos...
  > Procesados 21000 documentos...
  > Procesados 22000 documentos...
  > Procesados 23000 documentos...
  > Procesados 24000 documentos...
  > Procesados 25000 documentos...
  > Procesados 26000 documentos...
  > Procesados 27000 docume

KeyboardInterrupt: 